# Setup environment

In [ ]:
# !pip install transformers==4.45.2  datasets==3.0.1   torch-summary  jiwer==3.0.4 torchaudio   wandb >ou 


In [ ]:
# !pip install https://github.com/kpu/kenlm/archive/master.zip >ou

In [ ]:
# !pip install pyctcdecode==0.5.0  evaluate==0.4.3 >ou

In [ ]:

from dataclasses import dataclass, field
from tqdm import  tqdm
# import wandb

import torch
from torch.utils.data import  DataLoader
from torch import nn, optim,Tensor


# from torchsummary import summary
import torchaudio
import torchaudio.transforms as T


import transformers
from transformers import (
    Trainer,
    TrainingArguments,
    Wav2Vec2Processor,
    Wav2Vec2ForCTC,
    Wav2Vec2FeatureExtractor

)

from copy import deepcopy
from typing import Any, Dict, List, Optional, Union,Tuple
# import jiwer
from inference import Inference

import numpy as np

from evaluate import load
import IPython.display as ipd

from dataset import SemiSupervisedDataset,DatasetValidated,DataCollatorCTCWithPadding,DataCollatorTeacherCTCWithPadding
from tokenizer import Wav2Vec2WordpieceTokenizer
torchaudio.set_audio_backend('soundfile')

In [ ]:
name_version="Student_gen_4"
!wandb login --relogin <token>
%env WANDB_PROJECT=ASR_with_NST
wandb.init(project='ASR_with_NST', name=f"{name_version}")

In [ ]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", device)

In [ ]:
# model_name_student = "nguyenvulebinh/wav2vec2-base-vietnamese-250h"
model_name_teacher= "/kaggle/input/student-gen-3-subword"
model_name_student = "/kaggle/input/student-gen-4-subword"

batch_size = 40
batch_size_text = 2
repo_name="Student_base"
checkpoint_resume=model_name_student
# checkpoint_resume=None
# lr = 0.0005 * batch_size ** (1 / 2)
lr = 1e-5
max_lr=5e-5
max_epochs = 15



tokenizer=Wav2Vec2WordpieceTokenizer('/kaggle/input/vocab-model/vocab.json', bos_token="<s>", eos_token="</s>", unk_token="<unk>", pad_token="<pad>", word_delimiter_token="|")
feature_extractor=Wav2Vec2FeatureExtractor.from_pretrained("nguyenvulebinh/wav2vec2-base-vietnamese-250h")
processor =Wav2Vec2Processor(feature_extractor=feature_extractor,tokenizer=tokenizer)

In [ ]:

train_dataset = SemiSupervisedDataset(
    processor,
)
test_dataset=DatasetValidated(  
    processor,
    path='/kaggle/input/commonvoice-vie/cv-corpus-15.0-2023-09-08',
    path_csv='/kaggle/input/datacsv/commonvoice_test.csv'
)
val_dataset=DatasetValidated(  
    processor,
)

In [ ]:
teacher = Wav2Vec2ForCTC.from_pretrained(model_name_teacher)
teacher.to(device)
teacher.freeze_feature_encoder()
# print(model)
# print(summary(model,input_size= (1, 500)))

In [ ]:
teacher_inference=Inference(teacher,processor)

In [ ]:
data_collator = DataCollatorTeacherCTCWithPadding(processor=processor,teacher=teacher_inference, padding=True)
data_collator_test = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
# train_dataloader = DataLoader(
#     train_dataset,
#     batch_size=batch_size,
#     collate_fn=data_collator,
#     shuffle=True,
#     pin_memory=True,
#     num_workers=0,
#     drop_last=False,
# )
test_dataloader = DataLoader(
    test_dataset,
    batch_size=batch_size_text,
    collate_fn=data_collator_test,
    shuffle=False,
    pin_memory=True,
    num_workers=2,
    drop_last=False,
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=batch_size_text,
    collate_fn=data_collator_test,
    shuffle=False,
    pin_memory=True,
    num_workers=2,
    drop_last=False,
)

In [ ]:
len(train_dataset)

In [ ]:
# for d in tqdm(train_dataloader):
#     pass
#     break


In [ ]:
student = Wav2Vec2ForCTC.from_pretrained(model_name_student)
student.to(device)
student.freeze_feature_encoder()
# print(model)
# print(summary(model,input_size= (1, 500)))

In [ ]:
wer_metric = load("wer")
cer_metric = load("cer")
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
  
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    cer = cer_metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer,"cer":cer}

In [ ]:
config = {
    "model_teacher":model_name_student,
    "model_student":model_name_teacher,
    "learning_rate": lr,
    "max_epochs": max_epochs,
    "batch_size":  batch_size,
}
wandb.config = config

In [ ]:

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=False,
  per_device_train_batch_size=batch_size,
  per_device_eval_batch_size =batch_size_text,
  evaluation_strategy="steps",
  num_train_epochs=max_epochs,
  fp16=True,
  use_cpu=False,
  gradient_checkpointing=True, 
  save_steps=500,
  eval_steps=500,
  learning_rate=lr,
  weight_decay=0.01,
  do_train =True,
  save_total_limit=2,
  load_best_model_at_end=True,
  report_to="wandb", 
  run_name=f"{name_version}", 
  logging_steps=500,
  warmup_steps=50,
  logging_strategy='steps'
   )

In [ ]:
total_steps=int((len(train_dataset)//batch_size+1)*max_epochs)
optimizer = optim.AdamW(student.parameters(), lr=lr, betas=(0.9, 0.9999))
# scheduler = optim.lr_scheduler.OneCycleLR(
#     optimizer, max_lr=max_lr, pct_start=0.3, total_steps=total_steps
# )
scheduler = optim.lr_scheduler.ConstantLR(optimizer, factor=1.0, total_iters=total_steps)
print(scheduler.get_lr())
print(total_steps)

In [ ]:
trainer = Trainer(
    model=student,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=processor.feature_extractor,
    optimizers =[optimizer,scheduler]
#     optimizers =[optimizer,None]
)

In [ ]:
trainer.train(resume_from_checkpoint=checkpoint_resume)
wandb.finish()

In [ ]:
inference=Inference(student,processor)
running_wers=inference.test_wer(val_dataloader,50)
# running_wers=teacher_inference.test_wer(val_dataloader,100)
print(np.mean(running_wers))

In [ ]:
# running_wers=inference.test_wer(test_dataloader)
# print(np.mean(running_wers))